In [41]:
PLACE_POLYS_NP = '../data/place_polys_np.geojson'
import geopandas as gp
import sys, os
sys.path.insert(0, os.path.abspath('../'))
from utils.geofunc import grid_area, haversine
place_gpdf = gp.read_file(PLACE_POLYS_NP)
place_gpdf_small = place_gpdf[place_gpdf['radius+1km']<=32000]

places_small = zip(place_gpdf_small['place##cnt'].values,
                   place_gpdf_small.cntr.apply(eval).apply(lambda x: (x[1],x[0])).values,
                   place_gpdf_small['radius+1km'].apply(lambda x: '{}km'.format(int(x/1000)+1)).values)
place_gpdf_large = polys[polys['radius+1km']>32000]
place_large = get_place_large(place_gpdf_large)
places = places_small+place_large

In [42]:
def get_max_dis_from_center_to_ext(centr,ext_coords):
    lon1, lat1 = centr
    return max([haversine(lon1,lat1, lon2,lat2) for lon2, lat2 in ext_coords])

def get_cntr_radius(poly):
    cntr = poly.centroid.coords[0]
    ext_coords = poly.exterior.coords
    radius = get_max_dis_from_center_to_ext(cntr, ext_coords)
    return cntr, radius, ext_coords

def get_place_large(place_gpdf_large):
    import shapely.geometry as shpgeo
    l = zip(large['place##cnt'].values,large.geometry.values, large['radius+1km'])
    place_large = []
    for place, poly, radius in l:
        cnt = 0
        ngrid = int(radius/32000) +2 
        w,s,e,n = poly.bounds
        gridded = grid_area((s,w),(n,e), ngrid=ngrid)
        desire_boxes = []
        for (s,w),(n,e) in gridded:
            box = shpgeo.box(w,s,e,n)
            if box.intersects(poly):
                [lon,lat], radius, ext_coords = get_cntr_radius(box)
                sub_place = place + '[%d]' % cnt
                cnt+=1
                desire_boxes.append((sub_place, [lat,lon], '{}km'.format(int(radius/1000)+1)))
        place_large.extend(desire_boxes)
    return place_large